# Lightweight Fine-Tuning Project



TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

In [1]:
!pip install datasets peft -qq
!pip install accelerate -qq
!pip install bitsandbytes -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

# Import Packages

In [ ]:
from google.colab import userdata
from huggingface_hub import login
login()

# Steps HuggingFace mentioned in Readme anf requer file
# Access Key for Gemma Model (HuggingFace)



from datasets import load_dataset
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    Trainer,
    BitsAndBytesConfig)

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Load Model and Tokenizer

In [3]:
# Used Gemma 2 Foundation Model

model_name = "google/gemma-2-2b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
    )


tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    )

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# model.config.pretraining_tp = 1
# model.config.use_cache = False


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [4]:
# Print base model
print(base_model)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_

In [5]:
print(base_model.config)

Gemma2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google/gemma-2-2b",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 8,
  "num_hidden_layers": 26,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_

In [6]:
def trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    return f"- Trainable model parameters: {trainable_params}.\n- All model parameters: {all_param}.\n- Percentage of trainable model parameters: {100 * trainable_params / all_param:.2f}%"

print(trainable_parameters(base_model))

- Trainable model parameters: 590065920.
- All model parameters: 1602203904.
- Percentage of trainable model parameters: 36.83%


# Test Model before Fine Tuning

In [7]:
# Test model
# text = "Dedicated and innovative AR developer with a passion for creating engaging and interactive experiences. Skilled in Unity, ARKit, and ARCore. Collaborative team player with a strong background in computer science and software engineering."

pre_training_test_data = {
    "Headline": "Augmented Reality Developer | Creating Immersive Experiences",
    "About Me": "Dedicated and innovative AR developer with a passion for creating engaging and interactive experiences. Skilled in Unity, ARKit, and ARCore. Collaborative team player with a strong background in computer science and software engineering.",
    "Experience": "Senior AR Developer | Pixelloid | Zurich, Switzerland | 2020-2022 | Led the development of multiple AR projects for various clients, including a popular museum exhibit and a cutting-edge industrial training platform. Utilized Unity and ARKit to create immersive experiences that increased user engagement by 30%.; AR Developer | Breadwinner Studios | Berlin, Germany | 2018-2020 | Worked on several AR projects, including a mobile app for furniture shopping and a virtual try-on platform for fashion brands. Contributed to the development of in-house tools for tracking and optimizing AR experiences.; Junior AR Developer | Whimsy Tech | Amsterdam, Netherlands | 2016-2018 | Collaborated with a team to create AR experiences for several clients, including a interactive art installation and a 3D model visualization tool. Gained expertise in ARCore and Unity.",
    "Skills": "Unity; ARKit; ARCore; C#; Java; JavaScript; WebGL; 3D Modeling; Project Management",
    "Recommendations": "Benjamin Ryder | Lead Developer, Breadwinner Studios | Zachariah was an invaluable member of our AR development team. His expertise in ARKit and Unity was unparalleled, and his ability to communicate complex ideas to the team was exceptional. I highly recommend him for any AR development project."
}

# Create a prompt according to our task
pre_training_test_prompt = (
    "Profile Summary Task:\n"
    f"Headline: {pre_training_test_data['Headline']}\n"
    f"About Me: {pre_training_test_data['About Me']}\n"
    f"Experience (summarize key achievements and responsibilities): {pre_training_test_data['Experience']}\n"
    f"Skills: {pre_training_test_data['Skills']}\n"
    f"Recommendations (highlight strengths): {pre_training_test_data['Recommendations']}\n"
    "\n\nGenerate a concise profile summary:"
)
device = "cuda:0"
inputs = tokenizer(pre_training_test_prompt, return_tensors="pt").to(device)

outputs = base_model.generate(**inputs, max_new_tokens=512)
tokenizer.decode(outputs[0], skip_special_tokens=True)

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


"Profile Summary Task:\nHeadline: Augmented Reality Developer | Creating Immersive Experiences\nAbout Me: Dedicated and innovative AR developer with a passion for creating engaging and interactive experiences. Skilled in Unity, ARKit, and ARCore. Collaborative team player with a strong background in computer science and software engineering.\nExperience (summarize key achievements and responsibilities): Senior AR Developer | Pixelloid | Zurich, Switzerland | 2020-2022 | Led the development of multiple AR projects for various clients, including a popular museum exhibit and a cutting-edge industrial training platform. Utilized Unity and ARKit to create immersive experiences that increased user engagement by 30%.; AR Developer | Breadwinner Studios | Berlin, Germany | 2018-2020 | Worked on several AR projects, including a mobile app for furniture shopping and a virtual try-on platform for fashion brands. Contributed to the development of in-house tools for tracking and optimizing AR exper

# Load Dataset

In [8]:
# Load dataset
dataset = load_dataset("ilsilfverskiold/linkedin_profiles_synthetic", split='train')
dataset.features


README.md:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/232M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/225M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6904 [00:00<?, ? examples/s]

{'FirstName': Value(dtype='string', id=None),
 'LastName': Value(dtype='string', id=None),
 'Headline': Value(dtype='string', id=None),
 'Location': Value(dtype='string', id=None),
 'About Me': Value(dtype='string', id=None),
 'Experience': Value(dtype='string', id=None),
 'Education': Value(dtype='string', id=None),
 'Skills': Value(dtype='string', id=None),
 'Certifications': Value(dtype='string', id=None),
 'Recommendations': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'embeddings_nv-embed-v1': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'embeddings_nv-embedqa-e5-v5': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'embeddings_bge-m3': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'embeddings_arctic-embed-l': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'embeddings_mistral-7b-v2': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),

In [10]:
# Remove specified features

columns_to_remove = [
    'embeddings_nv-embed-v1',
    'embeddings_nv-embedqa-e5-v5',
    'embeddings_bge-m3',
    'embeddings_arctic-embed-l',
    'embeddings_mistral-7b-v2',
    'embeddings_gte-large-en-v1.5',
    'embeddings_text-embedding-ada-002',
    'embeddings_text-embedding-3-small',
    'embeddings_voyage-3',
    'embeddings_mxbai-embed-large-v1 ',
]

#Efficiently remove columns using Dataset.remove_columns
dataset = dataset.remove_columns(columns_to_remove)
dataset_subset = dataset.select(range(2500))
dataset_subset[0]

{'FirstName': 'Zachariah',
 'LastName': 'Witting',
 'Headline': 'Augmented Reality Developer | Creating Immersive Experiences',
 'Location': 'Zurich, Switzerland',
 'About Me': 'Dedicated and innovative AR developer with a passion for creating engaging and interactive experiences. Skilled in Unity, ARKit, and ARCore. Collaborative team player with a strong background in computer science and software engineering.',
 'Experience': 'Senior AR Developer | Pixelloid | Zurich, Switzerland | 2020-2022 | Led the development of multiple AR projects for various clients, including a popular museum exhibit and a cutting-edge industrial training platform. Utilized Unity and ARKit to create immersive experiences that increased user engagement by 30%.; AR Developer | Breadwinner Studios | Berlin, Germany | 2018-2020 | Worked on several AR projects, including a mobile app for furniture shopping and a virtual try-on platform for fashion brands. Contributed to the development of in-house tools for track

In [27]:
split_datasets = dataset_subset.train_test_split(test_size=0.2, seed=42)

train_dataset = split_datasets['train']
test_dataset = split_datasets['test']

In [12]:
def preprocess_function(batch):
    prompts = [
        "Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.\n\n"
        f"Headline: {headline}\n"
        f"About Me: {about_me}\n"
        f"Experience (summarize the key achievements and responsibilities): {experience}\n"
        f"Skills: {skills}\n"
        f"Recommendations (highlight the main strengths mentioned): {recommendations}\n"
        for headline, about_me, experience, skills, recommendations in zip(
            batch["Headline"],
            batch["About Me"],
            batch["Experience"],
            batch["Skills"],
            batch["Recommendations"]
        )
    ]
    print("Sample prompt:", prompts[0])
    return tokenizer(prompts, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

In [28]:
# Apply preprocessing and tokenization
tokenized_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

# Check the keys of the first entry
print(tokenized_dataset[0].keys())

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Sample prompt: Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.

Headline: Product Manager Data Solutions | Driving Business Growth through Data-Driven Insights
About Me: Highly motivated and results-driven product manager with a strong background in data analysis and business development. Proven track record of delivering data-driven solutions that drive business growth and improve customer experience. Skilled in data storytelling, business acumen, and team leadership.
Experience (summarize the key achievements and responsibilities): Product Manager, Data Solutions | Ericsson | Stockholm, Sweden | 2020-Present | Developed and executed product strategies for data analytics and business intelligence solutions, resulting in 25% increase in revenue growth. Led cross-functional teams to design and implement data-driven solutions, improving customer satisfaction by 30%.; Senior Business Analyst | Schneider Ele

In [26]:
print(tokenized_dataset[0])
print(tokenized_dataset.column_names)

{'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 24985, 573, 2412, 6790, 4691, 235269, 11941, 476, 66004, 8959, 13367, 62144, 2621, 33319, 235269, 7841, 235269, 578, 39711, 235265, 109, 152183, 235292, 173038, 47170, 42663, 1420, 40780, 75339, 1998, 89164, 108, 4398, 1842, 235292, 98233, 578, 21367, 5509, 26720, 675, 476, 11828, 604, 10241, 30509, 578, 29295, 12219, 235265, 180329, 575, 11823, 235269, 5509, 15647, 235269, 578, 5509, 5826, 235265, 111892, 2970, 5398, 675, 476, 3779, 3824, 575, 6875, 8042, 578, 6815, 13299, 235265, 108, 40840, 591, 203819, 889, 573, 2621, 33319, 578, 2418, 20114, 124

In [29]:
# Apply preprocessing and tokenization
test_tokenized_dataset = test_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

# Check the keys of the first entry
print(test_tokenized_dataset[0].keys())

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Sample prompt: Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.

Headline: AI Engineer | Developing AI solutions for Industry 4.0
About Me: Highly motivated AI engineer with 5+ years of experience in developing and deploying AI solutions for Industry 4.0. Skilled in deep learning, computer vision, and natural language processing. Strong background in software development and data science. Passionate about using AI to drive innovation and improve business outcomes.
Experience (summarize the key achievements and responsibilities): Senior AI Engineer, Ericsson (2020-Present) | Ericsson | Stockholm, Sweden | Developed and deployed AI-powered solutions for 5G network optimization and predictive maintenance. Led a team of engineers to integrate AI with existing infrastructure. Collaborated with cross-functional teams to drive business outcomes and improve customer experience. | 2020-2022; AI Engineer, NVIDIA (2

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

# Configure LoRA

In [31]:
# Configure Low-Rank Adaptation (LoRA) settings

lora_config = LoraConfig(
    r=8,                           # Rank of the low-rank decomposition
    lora_alpha=32,                 # Scaling factor for the LoRA updates
    lora_dropout=0.05,             # Dropout rate to apply to the LoRA updates
    bias="none",                   # Specify how to handle biases in LoRA
    target_modules=[               # List of module names to apply LoRA to
        'q_proj',                  # Query projection in attention mechanism
        'k_proj',                  # Key projection in attention mechanism
        'v_proj',                  # Value projection in attention mechanism
        'o_proj',                  # Output projection in attention mechanism
        'gate_proj',               # Gate projection
        'up_proj',                 # Up projection in feed-forward network
        'down_proj',               # Down projection in feed-forward network
        'lm_head',                 # Language model head
    ],
    task_type="CAUSAL_LM",         # Task type: Causal Language Modeling
)


In [32]:
# LoRA trainable version of model

model_name = "google/gemma-2-2b"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=False
#     )


tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    )

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


model = prepare_model_for_kbit_training(foundation_model)
model = get_peft_model(foundation_model, lora_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [33]:
# Trainable parameter count

print(trainable_parameters(model))

- Trainable model parameters: 12449792.
- All model parameters: 1614653696.
- Percentage of trainable model parameters: 0.77%


In [34]:
# Print Lora version of base model

print(f"PEFT Model: {model}")

PEFT Model: PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              

# Train Model

In [35]:
# Define the training arguments

training_args = TrainingArguments(
    output_dir="gemma-peft-linkedin",   # Output directory for the fine-tuned model
    learning_rate=2e-4,                 # Learning rate for the optimizer
    logging_steps=1,                    # Log training progress every 1 step
    max_steps=4,                        # Maximum number of training steps (set to 2 for testing)
    # optim="paged_adamw_8bit",         # Optimizer to use (8-bit AdamW optimizer)
    gradient_checkpointing=True,        # Enable gradient checkpointing to save memory
    save_steps=1,                       # Save a checkpoint every 1 step
    per_device_train_batch_size=1,      # Batch size for training (set to 1 for testing)
    remove_unused_columns=False,        # Eliminate the unused column
)

# Create the Trainer object
trainer = Trainer(
    model=model,                        # The pre-trained model to fine-tune
    train_dataset=tokenized_dataset,        # The training dataset
    args=training_args,                 # The training arguments
    data_collator=DataCollatorForLanguageModeling(
        tokenizer,
        mlm=False)                      # Data collator for language modeling (mlm=False means no masked language modeling)
)

In [ ]:
# Silence the warnings
model.config.use_cache = False

# Train model
trainer.train()

#Login wandb - API key

Step,Training Loss
1,1.867900
2,1.250100
3,1.740900
4,1.232300


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

TrainOutput(global_step=4, training_loss=1.5228076577186584, metrics={'train_runtime': 363.3177, 'train_samples_per_second': 0.011, 'train_steps_per_second': 0.011, 'total_flos': 25030258851840.0, 'train_loss': 1.5228076577186584, 'epoch': 0.002})

In [37]:
model.half()  # Convert model to half precision
model.to(device)

test_data = {
    "Headline": test_dataset[399]['Headline'],
    "About Me": test_dataset[399]['About Me'],
    "Experience": test_dataset[399]['Experience'],
    "Skills": test_dataset[399]['Skills'],
    "Recommendations": test_dataset[399]['Recommendations'],
}

# Create a prompt according to our task
test_prompt = (
    "Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.\n\n",
    f"Headline: {test_data['Headline']}\n"
    f"About Me: {test_data['About Me']}\n"
    f"Experience (summarize key achievements and responsibilities): {test_data['Experience']}\n"
    f"Skills: {test_data['Skills']}\n"
    f"Recommendations (highlight strengths): {test_data['Recommendations']}\n"
    "\n\nGenerate a concise profile summary:"
)

# Tokenize the prompt
inputs_token = tokenizer(
    test_prompt,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=512
).to(device)


model.eval()
with torch.no_grad():
    with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
        generated_ids = model.generate(**inputs_token, max_new_tokens=512)

# Decode the generated text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated Profile Summary:\n", generated_text)


<ipython-input-37-077d744bf027>:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):


Generated Profile Summary:
 Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.

<strong>Professional Summary:</strong>

Experienced and versatile professional with a strong background in digital marketing and social media management. Skilled in developing and executing effective social media strategies, leveraging data-driven insights to drive engagement and brand awareness. Passionate about staying up-to-date with the latest trends and technologies in the digital landscape.

<strong>Key Achievements:</strong>

* Increased brand awareness and engagement by 30% through the implementation of a comprehensive social media strategy.
* Developed and executed a successful influencer marketing campaign, resulting in a 20% increase in brand mentions and a 15% increase in website traffic.
* Managed a team of social media managers and interns, overseeing the development and execution of social media campaigns.

<stron

# Save Model

In [ ]:
# # # Renable warnings for inference
# model.config.use_cache = True

# # Save trained model
# trainer.model.save_pretrained("gemma-peft-linkedin", save_embedding_layers=True)

In [38]:
model.save_pretrained("./gemma-peft-linkedin")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [39]:
model.push_to_hub("gemma-peft-linkedin")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iyashnayi/gemma-peft-linkedin/commit/bd59dee08c12dbc827f2f8ed2a4c8dca5722f204', commit_message='Upload model', commit_description='', oid='bd59dee08c12dbc827f2f8ed2a4c8dca5722f204', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iyashnayi/gemma-peft-linkedin', endpoint='https://huggingface.co', repo_type='model', repo_id='iyashnayi/gemma-peft-linkedin'), pr_revision=None, pr_num=None)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [40]:
from peft import PeftConfig, PeftModel

MODEL_NAME = "google/gemma-2-2b"
PEFT_MODEL = "./gemma-peft-linkedin"

peft_config = PeftConfig.from_pretrained(PEFT_MODEL)
infe_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
infe_model = PeftModel.from_pretrained(model, PEFT_MODEL, torch_dtype=torch.float16, is_trainable=False)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0

In [46]:
 test_dataset[100]

{'FirstName': 'Ragnar',
 'LastName': 'Carlsson',
 'Headline': 'Quantitative Analyst | Data-Driven Insights for Investment Decisions',
 'Location': 'Stockholm, Sweden',
 'About Me': 'Results-driven quantitative analyst with 5+ years of experience in investment banking and asset management. Skilled in financial modeling, data analysis, and machine learning. Proven track record of delivering high-quality insights to drive business growth.',
 'Experience': 'UBS Investment Bank | Senior Quantitative Analyst | 2018-01-01 | 2020-12-31 | Developed and implemented advanced financial models to value complex assets and derivatives. Collaborated with trading and risk management teams to optimize portfolio performance and minimize risk exposure.; Nordea Asset Management | Quantitative Analyst | 2015-06-01 | 2018-12-31 | Designed and implemented data-driven investment strategies for various asset classes. Worked closely with portfolio managers to integrate quantitative insights into investment decis

In [47]:
# I am testing my own profile data to ensure its performance

test_data = {
    "Headline": test_dataset[100]['Headline'],
    "About Me": test_dataset[100]['About Me'],
    "Experience": test_dataset[100]['Experience'],
    "Skills": test_dataset[100]['Skills'],
    "Recommendations": test_dataset[100]['Recommendations'],
    }

# Create a prompt according to our task
test_prompt = (
    "Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.\n\n",
    f"Headline: {test_data['Headline']}\n"
    f"About Me: {test_data['About Me']}\n"
    f"Experience (summarize key achievements and responsibilities): {test_data['Experience']}\n"
    f"Skills: {test_data['Skills']}\n"
    f"Recommendations (highlight strengths): {test_data['Recommendations']}\n"
    "\n\nGenerate a concise profile summary:"
)

inputs = tokenizer(
    test_prompt,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=512
).to("cuda")


In [48]:
infe_model.eval()
with torch.no_grad():
    generated_ids_peft = infe_model.generate(**inputs, max_new_tokens=512)
peft_output = tokenizer.decode(generated_ids_peft[0], skip_special_tokens=True)
print("=== PEFT Model Output ===")
print(peft_output)


=== PEFT Model Output ===
Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.

<strong>Professional Summary</strong>

Experienced and highly motivated professional with a proven track record of success in the field of [insert relevant industry]. Demonstrated ability to develop and implement effective strategies to drive growth and profitability. Strong analytical skills and ability to identify and implement innovative solutions to address complex business challenges.

<strong>Key Achievements</strong>

* Successfully launched and managed a new product line, resulting in a 20% increase in revenue and a 15% reduction in operating costs.
* Developed and implemented a comprehensive marketing strategy that resulted in a 30% increase in customer base and a 25% increase in revenue.
* Led a team of 10 professionals to develop and implement a new business model that resulted in a 20% increase in revenue and a 15% red

In [49]:

base_model.eval()
with torch.no_grad():
    generated_ids_base = base_model.generate(**inputs, max_new_tokens=512)
base_output = tokenizer.decode(generated_ids_base[0], skip_special_tokens=True)
print("\n=== Base Model Output ===")
print(base_output)


=== Base Model Output ===
Given the following professional details, generate a concise profile summary highlighting key achievements, skills, and strengths.

<strong>Professional Summary</strong>

Experienced and highly motivated professional with a proven track record of success in the field of [insert relevant industry]. Demonstrated ability to develop and implement effective strategies to drive growth and profitability. Strong analytical skills and ability to identify and implement innovative solutions to complex business challenges.

<strong>Key Achievements</strong>

* Successfully launched and managed a new product line, resulting in a 20% increase in revenue and a 15% reduction in operating costs.
* Developed and implemented a new sales strategy that resulted in a 30% increase in sales volume and a 10% reduction in customer acquisition costs.
* Led a team of 10 sales representatives to achieve a 25% increase in sales volume and a 15% reduction in operating costs.

<strong>Skill

In [ ]:
# Push LoRA Adapters

trainer.push_to_hub("iyashnayi/gemma-peft-linkedin")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/2.14G [00:00<?, ?B/s]

events.out.tfevents.1740975172.43ea44dbb62e.1242.0:   0%|          | 0.00/6.32k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iyashnayi/gemma-ft/commit/c223e92f8f34e387cefa3a3983b5a67998cdcba4', commit_message='bmartinc80/gemma-ft', commit_description='', oid='c223e92f8f34e387cefa3a3983b5a67998cdcba4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iyashnayi/gemma-ft', endpoint='https://huggingface.co', repo_type='model', repo_id='iyashnayi/gemma-ft'), pr_revision=None, pr_num=None)

# Test Model after Fine Tuning

In [ ]:
# Merge Adapters with model

merged_model = model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:396: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications. You can opt to merge the adapter after cloning the weights (to untie the embeddings). You can untie the embeddings by loading the model with `tie_word_embeddings=False`. For example:
```python
from transformers import AutoModelForCausalLM

# Load original tied model
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", tie_word_embeddings=False)

# Set the randomly initialized lm_head to the previously tied embeddings
model.lm_head.weight.data = model.model.embed_tokens.weight.data.clone()

# Save the untied model
untied_model_dir = "dir/for/untied/model"
model.save_pretrained(untied_model_dir)
model.config.save_pretrained(untied_model_dir)

# Now use the original model but in untied format
model = AutoModelForCausalLM.from_pretrained(untied_model_dir)
`